In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

In [2]:
eur = Package("../datapackage.json")
scenarios = [
    {"model": "remind", "pathway": "SSP2-NPi", "year": 2020, "external scenarios": [{"scenario": "trade", "data": eur}]},
]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9",
        key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
        use_absolute_efficiency=True,
        #biosphere_name="ecoinvent-3.10-biosphere"
        
)

ndb.update("external")

premise v.(2, 1, 1, 'dev4')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

Processing scenarios:   0%|                     | 0/1 [00:00<?, ?it/s]

The following datasets will be duplicated:
+----------------------------------------------------+---------------------------+
|                        Name                        |     Reference product     |
+----------------------------------------------------+---------------------------+
| electricity production, natural gas, combined cycl | electricity, high voltage |
+----------------------------------------------------+---------------------------+


Processing scenarios: 100%|█████████████| 1/1 [00:36<00:00, 36.48s/it]

Done!



In [3]:
from premise.utils import load_database
for scenario in ndb.scenarios:
    scenario=load_database(scenario)

In [7]:
for scenario in ndb.scenarios:
    db = scenario["database"]
    regions = list(set([a["location"] for a in db if "TIMES" in a ["name"]]))
    suppliers_to_replace = [
        "market for electricity, high voltage",
        "market for electricity, medium voltage",
        "market for electricity, low voltage",
    ]
    for ds in db:
        if not any(x in ds["name"] for x in suppliers_to_replace):
            for e in ds["exchanges"]:
                if e["type"] == "technosphere":
                    if e["name"] in suppliers_to_replace and e["location"] in regions:
                        e["name"] += " (TIMES)"

In [8]:
ndb.write_db_to_brightway("eur 2020 5")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
---> MAJOR anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Title: Writing activities to SQLite3 database:
  Started: 07/10/2024 16:05:42
  Finished: 07/10/2024 16:06:04
  Total time elapsed: 00:00:21
  CPU %: 93.60
  Memory %: 12.23
Created database: eur 2020 5
Generate scenario report.
Report saved under /Users/romain/Github/eur-prospective-scenarios/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/eur-prospective-scenarios/dev.
